In [1]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
import os
from buildingmotif.utils import compile_model
from buildingmotif.namespaces import BMOTIF, BRICK, SH, RDFS

## Setup BuildingMOTIF

In [2]:
building_motif = BuildingMOTIF("sqlite://")
building_motif.setup_tables()

## Load model from file

In [3]:
medium_office_model = Model.create("http://example.org/building/")
medium_office_model.graph.parse(os.path.join('mediumOffice-validation', 'mediumOffice_brick_compiled.ttl'), format="ttl")

<Graph identifier=9a898aa7-3906-4ebc-a2b0-b94521ccc576 (<class 'rdflib.graph.Graph'>)>

In [4]:
brick = Library.load(ontology_graph="../libraries/brick/Brick-full.ttl")
ashrae_g36 = Library.load(directory="../libraries/ashrae/guideline36/")
manifest = Library.load(directory="mediumOffice-validation/constraints/")

2022-08-05 14:40:58,657 | root |  WARNING: Warning: could not find dependee http://www.w3.org/2001/XMLSchema#string
2022-08-05 14:40:58,659 | root |  WARNING: Warning: could not find dependee http://www.w3.org/2001/XMLSchema#string
2022-08-05 14:40:58,661 | root |  WARNING: Warning: could not find dependee http://www.w3.org/2001/XMLSchema#string
2022-08-05 14:40:58,663 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject
2022-08-05 14:40:58,666 | root |  WARNING: Warning: could not find dependee http://www.w3.org/2001/XMLSchema#string
2022-08-05 14:40:58,668 | root |  WARNING: Warning: could not find dependee http://www.w3.org/2001/XMLSchema#string
2022-08-05 14:40:58,670 | root |  WARNING: Warning: could not find dependee http://www.w3.org/2001/XMLSchema#string
2022-08-05 14:40:58,672 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice
2022-08-05 14:40:58,674 | root |  WARNING: Warning: 

## Validate that model passes constraints

In [5]:
shape_collections = [brick.get_shape_collection(), manifest.get_shape_collection(), ashrae_g36.get_shape_collection()]
validation_context = medium_office_model.validate(shape_collections)


In [6]:
print(f"Model is valid: {validation_context.valid}")

print("Reasons why:")
for diff in validation_context.diffset:
    print(" -" + diff.reason())

Model is valid: False
Reasons why:
 -n1a5bc69f0aa34d9097ad01e0f4e17e3ab1 needs between 1 and None uses of path https://brickschema.org/schema/Brick#deprecatedInVersion
 -http://example.org/building/5-Zone-PVAV needs between 1 and None instances of https://brickschema.org/schema/Brick#Supply_Air_Temperature_Setpoint on path https://brickschema.org/schema/Brick#hasPoint
 -n1a5bc69f0aa34d9097ad01e0f4e17e3ab2 needs between 1 and None uses of path https://brickschema.org/schema/Brick#deprecationMitigationMessage
 -n1a5bc69f0aa34d9097ad01e0f4e17e3ab3 needs between 1 and None uses of path https://brickschema.org/schema/Brick#deprecationMitigationMessage
 -n1a5bc69f0aa34d9097ad01e0f4e17e3ab2 needs between 1 and None uses of path https://brickschema.org/schema/Brick#deprecatedInVersion
 -http://example.org/building/5-Zone-PVAV-1 needs between 1 and None instances of https://brickschema.org/schema/Brick#Supply_Air_Temperature_Setpoint on path https://brickschema.org/schema/Brick#hasPoint
 -n1a5b

## Test to see which G36 AHU fault are supported by the AHUs in our model
We select the shapes of type `Analytics_Application` from the ASHRAE Guideline 36 Library. We then test our AHUs against each application. From the output of the test we can see which we support out of the box with our metadata model. We can also look at what we are missing in cases where we don't have the required entities.

In [11]:
shape_collections = [brick.get_shape_collection(), ashrae_g36.get_shape_collection()]
shapes_to_test = ashrae_g36.get_shape_collection().get_shapes_of_definition_type(BMOTIF["Analytics_Application"])
conformance = medium_office_model.test_model_against_shapes(
    shape_collections=shape_collections,
    shapes_to_test=shapes_to_test,
    target_class=BRICK["AHU"])
for shape_uri, validation_context in conformance.items():
    _, _, label = next(ashrae_g36.get_shape_collection().graph.triples((shape_uri, RDFS["label"], None)))
    print(f"Fault Condition: {label}")
    print(f"\t Conformance: {validation_context.valid}")
    if not validation_context.valid:
        print("\t Reasons why:")
        for diff in validation_context.diffset:
            print(f"\t- {diff.reason()}")


Fault Condition: MAT too low; should be between OAT and RAT
	 Conformance: True
Fault Condition: MAT too high; should be between OAT and RAT
	 Conformance: True
Fault Condition: Too many changes in Operating State
	 Conformance: False
	 Reasons why:
	- http://example.org/building/5-Zone-PVAV-2 needs between 1 and None instances of https://brickschema.org/schema/Brick#Operating_Mode_Status on path https://brickschema.org/schema/Brick#hasPoint
	- http://example.org/building/5-Zone-PVAV needs between 1 and None instances of https://brickschema.org/schema/Brick#Operating_Mode_Status on path https://brickschema.org/schema/Brick#hasPoint
	- http://example.org/building/5-Zone-PVAV-1 needs between 1 and None instances of https://brickschema.org/schema/Brick#Operating_Mode_Status on path https://brickschema.org/schema/Brick#hasPoint
Fault Condition: SAT too low; should be higher than MAT
	 Conformance: True
Fault Condition: OA fraction is too low or too high; shoud equal %OA_min
	 Conformance: 

'Https://Nrel.Gov/Buildingmotif#Analytics_Application'

## Fix Model

In [ ]:
# Model is missing feeds from 1 AHU to 1 VAV

# 1 VAV is missing Discharge_Air_Flow_Sensor
# apply g36 template to that VAV

## Validate and pass

In [ ]:
triples = ashrae_g36.get_shape_collection().graph.triples((None, SH["property"], None))
for s, p, o in triples:
    print(f"{s} {p} {o}")

## Compile Model against Brick

In [ ]:
model_graph = compile_model(medium_office_model, [brick.get_shape_collection()])

In [ ]:
model_graph.serialize("mediumOffice-validation/mediumOffice_brick_compiled.ttl", format="ttl")